# GO2 Learning to stand via genetic algo 

## Setup

In [ ]:
import numpy as np
import mujoco
import mujoco.viewer
import mediapy as media
from robot_descriptions import go2_mj_description
import torch
import torch.nn as nn

### Import Model

Let's take a look at the model xml

In [ ]:
with open(go2_mj_description.MJCF_PATH, 'r') as file:
    model_xml = file.read()
print(model_xml)

print(go2_mj_description.MJCF_PATH)

In [ ]:
model = mujoco.MjModel.from_xml_path(go2_mj_description.PACKAGE_PATH + "/scene.xml")
data = mujoco.MjData(model)

renderer = mujoco.Renderer(model)

## Experimentation

First up, lets render the scene

In [ ]:
duration = 3.8  # (seconds)
framerate = 60  # (Hz)

frames = []
mujoco.mj_resetData(model, data)  # Reset state and time.
while data.time < duration:
  mujoco.mj_step(model, data)
  if len(frames) < data.time * framerate:
    renderer.update_scene(data)
    pixels = renderer.render()
    frames.append(pixels)
media.show_video(frames, fps=framerate)

Plan is to create a neural network to control the robot. 
What are the inputs?
Robot needs to be aware of state (joint angles, joint velocities)

See https://mujoco.readthedocs.io/en/stable/python.html#named-access

The model has names associated with joints (e.g. `<joint name="FR_hip_joint" class="abduction"/>`).  
Named joints can be accessed on the `mujoco.MjData` object via `data.jnt("FR_hip_joint")`.

In [ ]:
joint_names = [mujoco.mj_id2name(model, mujoco.mjtObj.mjOBJ_JOINT, i) for i in range(1, model.njnt)]
print("Joint Names:", joint_names)
for joint_name in joint_names:
    print(f"{joint_name}: qpos: {data.jnt(joint_name).qpos}, qvel: {data.jnt(joint_name).qvel}")

The network should output the joint velocities for each joint.

In [ ]:
frames = []
mujoco.mj_resetData(model, data)  # Reset state and time.
while data.time < duration:
  mujoco.mj_step(model, data)
  if len(frames) < data.time * framerate:
    data.jnt("FL_thigh_joint").qvel = 2
    renderer.update_scene(data)
    pixels = renderer.render()
    frames.append(pixels)
media.show_video(frames, fps=framerate)

What about the score?  
For now, let's do final z-height of the bot after 3 seconds. (Bonus if it learns to jump!)
Let's have a look at the body "base".

In [ ]:
print(data.body("base"))

looks like xpos[2] is what we need!  
Importand note from docs: 
> The bindings include Python classes that expose MuJoCo data structures. For maximum performance, these classes provide access to the raw memory used by MuJoCo without copying or buffering. This means that some MuJoCo functions (e.g., mj_step) change the content of fields in place. The user is therefore advised to create copies where required.

https://mujoco.readthedocs.io/en/stable/python.html#structs

Now running the previous code again, we should see the z-height of the bot drop as the bot falls to the ground.  

In [ ]:
frames = []
mujoco.mj_resetData(model, data)
while data.time < duration:
  mujoco.mj_step(model, data)
  if len(frames) < data.time * framerate:
    data.jnt("FL_thigh_joint").qvel = 2
    renderer.update_scene(data)
    print(data.body("base").xpos[2])
    pixels = renderer.render()
    frames.append(pixels)
media.show_video(frames, fps=framerate)

# Defining the network
Okay so the we want to define a network that maps current joint angles and velocities to joint velocities.  
Let's also add the z-height of the bot as an input to give it awareness of it's own score.  


N: (qpos, qvel, z-height) -> qvel

In [ ]:
class Network(nn.Module):
    def __init__(self, input_size, hidden_size, output_size):
        super(Network, self).__init__()
        self.fc1 = nn.Linear(input_size, hidden_size)
        self.relu = nn.ReLU()
        self.fc2 = nn.Linear(hidden_size, hidden_size)
        self.fc3 = nn.Linear(hidden_size, output_size)

    def forward(self, x):
        out = self.fc1(x)
        out = self.relu(out)
        out = self.fc2(out)
        out = self.relu(out)
        out = self.fc3(out)
        return out

hidden_size = 64
network = Network(2*len(joint_names) + 1, hidden_size, len(joint_names))

network

Let's create another class that can act as an interface between the mujoco data and the network

In [ ]:
class Go2Interface():
    def __init__(self, model, network):
        self.model = model
        self.network = network

    def step(self):
        qpos = [data.jnt(joint_name).qpos for joint_name in joint_names]
        qvel = [data.jnt(joint_name).qvel for joint_name in joint_names]
        z_height = [data.body("base").xpos[2]]
        qpos_qvel = torch.tensor(np.concatenate([qpos, qvel, [z_height]]), dtype=torch.float32).T
        qvel = self.network(qpos_qvel).detach().numpy()[0]
        for joint_name, vel in zip(joint_names, qvel):
            data.jnt(joint_name).qvel = vel

go2_interface = Go2Interface(model, network)

Let's try run the randomly initialized network on the joint angles and velocities

In [ ]:
frames = []
mujoco.mj_resetData(model, data)
while data.time < duration:
  mujoco.mj_step(model, data)
  if len(frames) < data.time * framerate:
    go2_interface.step()
    renderer.update_scene(data)
    pixels = renderer.render()
    frames.append(pixels)
media.show_video(frames, fps=framerate)

Nice!

## Genetic Algorithm
Now we have the network, it is time to create the genetic algorithm.  
At each generation, we want to simulate a population of networks and evaluate them on a fitness function.  
The fitness function is simply the z-height of the bot after 3 seconds.  